In [1]:
import numpy as np
from scipy.stats import levy_stable

In [4]:
def prior_sample():
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    alpha = uniform.rvs(1.1, 2)  # U[1.1, 2]
    beta = uniform.rvs(-1, 1)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 300)  # U[-300, 300]
    return [alpha, beta, gamma, delta]

In [5]:
def univ_alpha_stable_sampler(params,size,seed) :
    alpha,beta,gamma,delta = params[0],params[1],params[2],params[3]
    #print(alpha,beta,gamma,delta)
    y_bar = 0
    np.random.seed(seed)
    w = np.random.standard_exponential(size=size)
    u = np.random.uniform(low = -np.pi/2, high = np.pi/2, size = size)
    if alpha == 1 :
        y_bar = 2/np.pi*((np.pi/2+beta*u)*np.tan(u)-beta*np.log((np.pi/2*w*np.cos(u))/(np.pi/2+beta*u)))
        return gamma*y_bar + delta
    else :
        S = (1+beta**2*np.tan(np.pi*alpha/2)**2)**(1/(2*alpha))
        B = 1/alpha*np.arctan(beta*np.tan(np.pi*alpha/2))
        #print(S,B)
        y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
        return gamma*y_bar + delta

In [9]:
def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

In [2]:
def summary_statistics(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    # For univariate alpha-stable models, use quantile-based estimators along with the mean of the data
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return [v_alpha, v_beta, v_gamma, mean_x]

In [ ]:
def smoothing_kernel(y, x, epsilon, covariance):
    # Define smoothing kernel using a Gaussian kernel with estimated covariance
    return multivariate_normal.pdf(y, mean=x, cov=epsilon**2 * covariance)


In [ ]:
def pi_LF(theta,y) :
    """give the approximate density pi_LF

    Args:
        theta (_type_): _description_
        y (_type_): _description_
    """
    [K((summary_statistics(y)-summary_statistics(x_i))/epsilon)/epsilon for x_i in x]
    

In [ ]:
def SMC_sampler(T=10,N =1000,n=200,covariance = covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    epsilons = np.linspace(0.1, 0.01, T)  # Tolerance schedule
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    theta = [prior_sample() for _ in range(N)]
    print("theta : \n", theta[:5])
    # set weights their initial values :
    #prior = np.array([1/(2-1.1),1/2,1/300,1/600])
    x = [univ_alpha_stable_sampler(params=theta_i, size=n, seed=seed) for theta_i in theta]
    weights = pi_LF(theta,y)/prior

